In [1]:
from glob import glob
from tqdm import tqdm
import re
import random
import matplotlib.pyplot as plt
import numpy as np
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Dropout, Conv2D, MaxPooling2D, Flatten,Activation,BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.losses import CategoricalCrossentropy

from tensorflow.keras.applications.efficientnet import preprocess_input,EfficientNetB0

from tensorflow.keras import callbacks as cb
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth( device=gpu, enable=True)

In [2]:
all_classes = glob('../data/orl_faces/*')
source_classes,target_classes=train_test_split(all_classes,test_size=0.1)
sorce_len=len(source_classes)
target_len=len(target_classes)
print(f"total {len(all_classes)} classes=source {sorce_len} + target {target_len} classes")

total 40 classes=source 36 + target 4 classes


In [3]:
W,H,CH = 64,64,1
def read_pgm(filename):
    byteorder='>'
    #first we read the image, as a raw file to the buffer
    with open(filename, 'rb') as f:
        buffer = f.read()
    
    #using regex, we extract the header, width, height and maxval of the image
    header, width, height, maxval = re.search(
        b"(^P5\s(?:\s*#.*[\r\n])*"
        b"(\d+)\s(?:\s*#.*[\r\n])*"
        b"(\d+)\s(?:\s*#.*[\r\n])*"
        b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    
    #then we convert the image to numpy array using np.frombuffer which interprets buffer as one dimensional array
    return np.frombuffer(buffer,
                            dtype='u1' if int(maxval) < 256 else byteorder+'u2',
                            count=int(width)*int(height),
                            offset=len(header)
                            ).reshape((int(height), int(width)))
def load_img(path, width = W):
    img = read_pgm(path.numpy().decode()).astype(np.float32)/255.
    shape_dst = np.min(img.shape[:2])
    oh = (img.shape[0] - shape_dst) // 2
    ow = (img.shape[1] - shape_dst) // 2
    center_square = np.array([width,width])// 2
    new_size=(width,width)
    
    # cropping + resize
    img = img[oh:oh + shape_dst, ow:ow + shape_dst]
    img=np.expand_dims(cv2.resize(img, new_size),-1)
    return tf.constant(img)
SUFFIX='.pgm'

In [4]:
WAYS = 3
SHOTS=5
QUERIES=2
BATCH_SIZE=4

In [5]:
min([len(glob(sdir+'/*'+SUFFIX)) for sdir in source_classes])

10

In [6]:
## exclude classes with too few examples
source_classes=[sdir for sdir in source_classes if len(glob(sdir+'/*'+SUFFIX))>SHOTS+QUERIES]

sorce_len=len(source_classes)
print(f"source {sorce_len} + target {target_len} classes")

source 36 + target 4 classes


In [7]:
min([len(glob(sdir+'/*'+SUFFIX)) for sdir in source_classes])

10

In [22]:
map_fun=lambda string: tf.py_function(func=load_img,inp=[string], Tout=tf.float32)
source_sub = [
    tf.data.Dataset.list_files(sc+'/*'+SUFFIX, shuffle=True)
    .map(map_fun)
    for sc in source_classes
]
target_sub = [
    tf.data.Dataset.list_files(sc+'/*'+SUFFIX, shuffle=True)
    .map(map_fun)
    for sc in target_classes
]
def gen(all_sub):
    supports = []
    querys = []
    order=np.random.permutation(len(all_sub))
    for tasks in range(len(all_sub)//WAYS):
        picked=[all_sub[tt] for tt in order[WAYS*tasks:WAYS*(tasks+1)]]
        support = tf.concat(
            [
                next(
                    iter(
                        sub.batch(SHOTS).prefetch(SHOTS)
                    )
                    ) for sub in picked
            ]
            , axis=0)
        idxs=np.random.choice(range(WAYS), size=QUERIES, replace=False)
        query = tf.concat(
            [
                next(
                    iter(
                        picked[idx].batch(1).prefetch(1)
                    )
                    ) for idx in idxs
            ]
            , axis=0)
        yield tf.concat([support, query], axis=0), tuple([keras.utils.to_categorical(idx,num_classes=WAYS) for idx in idxs])
def gen_source():
    return gen(source_sub)
def gen_target():
    return gen(target_sub)


data_source = tf.data.Dataset.from_generator(gen_source,
                                    output_types=(tf.float32,tuple([tf.float32]*QUERIES)),
                                    output_shapes=((WAYS*SHOTS+QUERIES,W,H,CH),tuple([WAYS]*QUERIES))).batch(BATCH_SIZE)
data_target = tf.data.Dataset.from_generator(gen_target,
                                    output_types=(tf.float32,tuple([tf.float32]*QUERIES)),
                                    output_shapes=((WAYS*SHOTS+QUERIES,W,H,CH),tuple([WAYS]*QUERIES))).batch(BATCH_SIZE)

In [9]:
def conv_net(input_shape):
    convnet = Sequential()
    for i in range(4):
        convnet.add(Conv2D(64,(3,3),padding='valid',input_shape=input_shape))
        convnet.add(BatchNormalization())
        convnet.add(Activation('relu'))
        convnet.add(MaxPooling2D())
    convnet.add(Flatten())
    return convnet
# def pretrain_net(input_shape):
#     base_model = EfficientNetB0(input_shape=(W,H,CH),weights="imagenet", include_top=False)
#     x_in=Input(shape=(W,H,CH))
#     x=preprocess_input(x_in)
#     out=Flatten()(base_model(x))
#     model = Model(inputs=x_in, outputs=out)
#     return model
def euclidean_distance(f_1,f_2):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(f_1 - f_2), axis=-1))

In [11]:
base_dim = (W,H,CH)
base_network = conv_net(base_dim)
# Query feature
x_in=Input(shape=(WAYS*SHOTS+QUERIES,W,H,CH))
latent_s=[base_network(x_in[:,ii]) for ii in range(WAYS*SHOTS)]
latent_q=[base_network(x_in[:,WAYS*SHOTS+ii]) for ii in range(QUERIES)]

y=list()
for qq in range(QUERIES):
    dist_scores=list()
    for ww in range(WAYS):
        latent_proto=tf.reduce_mean(tf.stack(latent_s[ww*SHOTS:(ww+1)*SHOTS],axis=-1),axis=-1)
        dist_avg=euclidean_distance(latent_q[qq],latent_proto)
        dist_scores.append(dist_avg)
    y.append(tf.nn.softmax(-tf.stack(dist_scores,axis=-1),axis=-1))

model = Model(inputs=x_in, outputs=tuple(y))

In [12]:
lr=0.001
def scheduler(epoch):
    global lr
    if epoch % 3 == 0:
        lr /= 2
    return lr
reduce_lr = cb.ReduceLROnPlateau(monitor='loss', factor=0.4,patience=2, min_lr=1e-8)
lr_sched = cb.LearningRateScheduler(scheduler)
tensorboard = cb.TensorBoard()
opt = tf.keras.optimizers.Adam(lr=0.002)
model.compile(loss=CategoricalCrossentropy(), optimizer=opt, metrics=['categorical_accuracy'])

In [13]:
# %%time
model.fit(data_source, epochs=1000, verbose=1,workers=4, callbacks=[reduce_lr,lr_sched, tensorboard],validation_data=)

Epoch 1/1000
      1/Unknown - 0s 93us/step - loss: 0.1186 - tf_op_layer_Softmax_loss: 0.0777 - tf_op_layer_Softmax_1_loss: 0.0409 - tf_op_layer_Softmax_categorical_accuracy: 1.0000 - tf_op_layer_Softmax_1_categorical_accuracy: 1.0000WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
3/3 [==============================] - 1s 360ms/step - loss: 0.1803 - tf_op_layer_Softmax_loss: 0.0587 - tf_op_layer_Softmax_1_loss: 0.1215 - tf_op_layer_Softmax_categorical_accuracy: 1.0000 - tf_op_layer_Softmax_1_categorical_accuracy: 1.0000
Epoch 2/1000
3/3 [==============================] - 1s 257ms/step - loss: 0.2746 - tf_op_layer_Softmax_loss: 0.1840 - tf_op_layer_Softmax_1_loss: 0.0905 - tf_op_layer_Softmax_categorical_accuracy: 1.0000 - tf_op_layer_Softmax_1_categorical_accu

KeyboardInterrupt: 